## Generative AI Safeguards: PII

<b>By:</b> <a href='https://www.evolveailabs.com/'>Evolve AI Labs</a><br>

This notebook aims to illustrate how to monitor PII data leakage in Generative AI. It is of paramount importance to safeguard <a href='https://en.wikipedia.org/wiki/Personal_data'>Personally Identifiable Information</a> while prompt injection techniques are evolving at a rapid pace. Intervention for PII data leak should proactive and not reactive as this kind of information has much higher impact on safety, trust and revenue of an organization. This notebook showcases how to identify if the response from an LLM contains PII information and how to intervene before the information reaches the end user.<br> <br>

## Libraries
- <a href='https://github.com/EdyVision/pii-codex'>PII Codex</a>: This library can aid in identifying PII information like email, phone numbers, location, IP addresses etc in free flowing text. This library uses <a href='https://spacy.io/'>spacy</a> models so it can run on the client side.
- <a href='https://www.langchain.com/'>Langchain</a>: Once the page source is available, we will use Langchain API to send the HTML to an LLM. Langchain is a popular and easy to use python tool for building LLM applications.

In [1]:
!pip install pii-codex
!pip install pii-codex[detections]
!python -m spacy download en_core_web_lg
!pip install langchain
!pip install openai

2024-01-22 04:20:19.493916: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-22 04:20:19.493986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-22 04:20:19.496052: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-22 04:20:19.507860: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-22 04:20:21.120317: W tensorflow/compiler/tf2

In [2]:
from google.colab import userdata
OPENAI_KEY = userdata.get('OPENAI_KEY')

In [3]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain import OpenAI
from pii_codex.services.analysis_service import PIIAnalysisService
from langchain.chat_models import ChatOpenAI
import pandas as pd

pii_analysis_service = PIIAnalysisService()

## LLM 
We will be using the <a href='https://openai.com/gpt-4'>GPT 4</a> LLM provided by <a href='https://openai.com/'>OpenAI</a> to illustrate an example.

In [4]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = ChatOpenAI(openai_api_key=OPENAI_KEY, model='gpt-4')
llm_chain = LLMChain(prompt=prompt, llm=llm)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
question = "What would Sam Altman reply if his executive assistant asked for his phone number?"

response = llm_chain.run(question)
response

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"Firstly, as an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. Therefore, I don't know what Sam Altman's phone number is. Furthermore, it's important to note that sharing such private information is against ethical guidelines. Even if I had that information, which I don't, I wouldn't be able to share it."

## Identifying PII
Once we have the response from the LLM, we can use ```analyze_item``` from ```pii_analysis_service``` to evaluate the response of the Language Model. This function will generate an analysis which can be checked for the existence of general or a particular type of PII.

In [6]:
pii_check = pii_analysis_service.analyze_item(response)
pii_check.to_dict()

{'analysis': [{'pii_type_detected': 'PERSON',
   'risk_level': 3,
   'risk_level_definition': 'Identifiable',
   'cluster_membership_type': 'Financial Information',
   'hipaa_category': 'Protected Health Information',
   'dhs_category': 'Linkable',
   'nist_category': 'Directly PII',
   'entity_type': 'PERSON',
   'score': 0.85,
   'start': 231,
   'end': 241}],
 'index': 0,
 'risk_score_mean': 3,
 'sanitized_text': "Firstly, as an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. Therefore, I don't know what <REDACTED>'s phone number is. Furthermore, it's important to note that sharing such private information is against ethical guidelines. Even if I had that information, which I don't, I wouldn't be able to share it."}

## Intervention
Below snippet shows the intervention check for phone numbers in Language Model responses.

In [7]:
if('PHONE_NUMBER' in list(set([analysis['pii_type_detected'] for analysis in pii_check.to_dict()['analysis']]))):
    print('Potential PII Leak')
else:
    print('Safe')

Safe


## Conclusion
This is how we can monitor PII data leakage in Generative AI using python libraries and avoid potential security threats.